In [24]:
import os
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly import offline
import plotly.io as pio
import plotly.express as px
from datetime import datetime, timedelta

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
from shapely.geometry import  Point, MultiPoint
import geopandas as gpd

from shapely import wkt

In [2]:
files = ['august.csv', 'september.csv', 'october.csv'] 
PATH ='../data/private/csv/fam/'

df = [pd.read_csv(os.path.join(PATH, file)) 
      for file in files]

df = pd.concat(df, sort=False)
print('Total de registros: %s'%df['month'].count())

query = '(month == 8 and day >= 27) or (month == 9) or (month == 10 and day <= 7)'
df = df.query(query)
print('Total de registros: %s'%df['month'].count())

df.head()

Total de registros: 53160
Total de registros: 26218


,time,sysclass,lat,lon,dir,vel,size,ttyyyxx1,ttyyyxx2,ttyyyxx3,...,maxz,meanvil,ttvil,meanprec,maxprec,year,month,day,hour,minute
12871,0.0,0.0,-2.75,-58.57,-999.0,0.0,13.0,0.0,0.0,0.0,...,22.28,0.18,2.31,0.97,1.18,2014,8,27,2,24
12872,0.2,1.0,-2.73,-58.57,0.0,11.1,30.0,0.0,0.0,0.0,...,28.17,0.30,9.10,1.80,4.16,2014,8,27,2,24
12873,0.0,0.0,-2.73,-58.57,-999.0,0.0,21.0,0.0,0.0,0.0,...,23.01,0.18,3.76,1.02,1.32,2014,8,27,3,0
12874,0.2,1.0,-2.75,-58.57,180.0,11.1,13.0,0.0,0.0,0.0,...,22.28,0.18,2.28,0.97,1.09,2014,8,27,3,0
12875,0.0,0.0,-1.71,-59.63,-999.0,0.0,12.0,0.0,0.0,0.0,...,21.46,0.16,1.96,0.92,0.97,2014,8,27,3,48


In [27]:
columns = ['lon', 'lat', 'ttyyyxx3', 'riverfrac', 'convfrac',
       'strafrac', 'meanz', 'maxz', 'meanvil', 'ttvil', 'meanprec', 'maxprec'
]

df_ = df[columns]
# df_ = df_[df_['ttyyyxx3'] > 0]
df_.head()

,lon,lat,ttyyyxx3,riverfrac,convfrac,strafrac,meanz,maxz,meanvil,ttvil,meanprec,maxprec
12871,-58.57,-2.75,0.0,0.0,0.00,100.00,21.46,22.28,0.18,2.31,0.97,1.18
12872,-58.57,-2.73,0.0,0.0,13.33,86.67,24.83,28.17,0.30,9.10,1.80,4.16
12873,-58.57,-2.73,0.0,0.0,0.00,100.00,21.46,23.01,0.18,3.76,1.02,1.32
12874,-58.57,-2.75,0.0,0.0,0.00,100.00,21.46,22.28,0.18,2.28,0.97,1.09
12875,-59.63,-1.71,0.0,75.0,0.00,100.00,20.53,21.46,0.16,1.96,0.92,0.97


In [28]:
grouped = df_.groupby(['lon', 'lat']).agg({'ttyyyxx3': 'sum', 'maxz': 'sum', 'ttvil': 'sum'})
grouped.head()

ttyyyxx3   maxz  ttvil
lon    lat                          
-61.59 -3.21      24.0  31.51   8.93
       -3.19       0.0  63.63  27.72
       -3.17       0.0  31.72   6.86
       -3.15       0.0  35.23  22.57
       -3.13       0.0  26.28   8.79

In [41]:
points = {
    'ttyyyxx3': [],
    'maxz': [],
    'ttvil': [],
    'geometry': []
}
for i, row in grouped.iterrows():
    x, y = i[0], i[1]
    point = Point(x, y)
    points['geometry'].append(point)
    points['ttyyyxx3'].append(int(row['ttyyyxx3']))
    points['maxz'].append(row['maxz'])
    points['ttvil'].append(row['ttvil'])   

In [47]:
if not os.path.exists('points'):
    os.makedirs('points')
gdf = gpd.GeoDataFrame(points)
gdf.crs = {'init' :'epsg:4326'}
gdf.to_file('points/lightnings.shp')

In [51]:
gdf.head()

,ttyyyxx3,maxz,ttvil,geometry
0,24,31.51,8.93,POINT (-61.59 -3.21)
1,0,63.63,27.72,POINT (-61.59 -3.19)
2,0,31.72,6.86,POINT (-61.59 -3.17)
3,0,35.23,22.57,POINT (-61.59 -3.15)
4,0,26.28,8.79,POINT (-61.59 -3.13)


In [49]:
am = gpd.read_file('../data/geodata/amazonas.shp')
am.head()

,CODMUN6,MUNIC_PIO,SEM_ACENTO,MUNIC_PI0,UF,UF_IBGE,REGI_O,MESO_IBGE,MESO_IBG0,MICRO_IBGE,MICRO_IB0,MESO_IBG1,MICRO_IB1,AREA_97,SEDE,SEDE0,N91,N96,CODCOMP,geometry
0,130020,ATALAIA DO NORTE,ATALAIA DO NORTE,1300201,AM,13,N,SUDOESTE AMAZONENSE,02,ALTO SOLIMOES,003,1302,130203,76355.390,-70.192,-4.372,8224.0,8108.0,130200300201,"POLYGON ((-73.800781 -7.111809, -73.7995899999..."
1,130165,GUAJARÁ,GUAJARA,1301654,AM,13,N,SUDOESTE AMAZONENSE,02,JURUA,004,1302,130204,8945.517,-72.584,-7.546,11714.0,10944.0,130200401654,"POLYGON ((-73.741705 -7.143248, -73.740475 -7...."
2,130180,IPIXUNA,IPIXUNA,1301803,AM,13,N,SUDOESTE AMAZONENSE,02,JURUA,004,1302,130204,13572.800,-71.695,-7.051,9837.0,11120.0,130200401803,"POLYGON ((-72.18244799999999 -7.577473, -72.18..."
3,130140,EIRUNEPÉ,EIRUNEPE,1301407,AM,13,N,SUDOESTE AMAZONENSE,02,JURUA,004,1302,130204,15878.120,-69.874,-6.660,21552.0,25776.0,130200401407,"POLYGON ((-71.50966699999999 -7.880687, -71.50..."
4,130060,BENJAMIN CONSTANT,BENJAMIN CONSTANT,1300607,AM,13,N,SUDOESTE AMAZONENSE,02,ALTO SOLIMOES,003,1302,130203,8704.716,-70.031,-4.383,19951.0,23633.0,130200300607,"POLYGON ((-71.34350499999999 -6.680839, -71.34..."


In [53]:
gdf_ = gdf[gdf['ttyyyxx3'] > 0]
gdf_.head()

,ttyyyxx3,maxz,ttvil,geometry
0,24,31.51,8.93,POINT (-61.59 -3.21)
12,2,46.80,61.68,POINT (-61.57 -3.37)
15,4,70.82,67.79,POINT (-61.57 -3.29)
22,1,108.04,88.63,POINT (-61.57 -3.09)
34,1,66.45,44.61,POINT (-61.55 -3.33)


In [140]:
statistics = {}
svil = {}
maxz = {}
for i, row in gdf_.iterrows():
    point = row['geometry']
    total = row['ttyyyxx3']
#     total2 = row['ttvil']
#     total3 = row['maxz']
    
    for i, city in am.iterrows():
        city_name = city['MUNIC_PIO']
        city_geom = city['geometry'] 
        if city_geom.contains(point):
            if not statistics.get(city_name):
                statistics[city_name] = 0
#                 svil[city_name] = 0
#                 maxz[city_name] = 0
            statistics[city_name] += total
#             svil[city_name] += total2
#             maxz[city_name] += total3
            break

In [68]:
def find(l, v):
    for i, x in enumerate(l):
        if v == x:
            return True, i
    return False, 0

In [139]:
labels = list(statistics.keys())
values = list(statistics.values())
# labels2 = list(svil.keys())
# values2 = list(svil.values())
# labels3 = list(maxz.keys())
# values3 = list(maxz.values())

lightnings = []
# vil = []
# maxz = []
for i, row in am.iterrows():
    city_name = row['MUNIC_PIO']
    found = False
    contains, index = 0, 0
    for label in labels:
        contains, index = find(labels, city_name)
        if contains:
            found = True
            break
    if found:    
        lightnings.append(values[index])
    else:
        lightnings.append(0)
        
am['lightnings'] = lightnings

In [83]:
am[am['lightnings'] > 0]

,CODMUN6,MUNIC_PIO,SEM_ACENTO,MUNIC_PI0,UF,UF_IBGE,REGI_O,MESO_IBGE,MESO_IBG0,MICRO_IBGE,...,MESO_IBG1,MICRO_IB1,AREA_97,SEDE,SEDE0,N91,N96,CODCOMP,geometry,lightnings
27,130320,NOVO AIRÃO,NOVO AIRAO,1303205,AM,13,N,NORTE AMAZONENSE,01,RIO NEGRO,...,1301,130101,37771.740,-60.944,-2.621,15279.0,14576.0,130100103205,"POLYGON ((-63.957748 -2.445123, -63.955813 -2....",7720
29,130010,ANORI,ANORI,1300102,AM,13,N,CENTRO AMAZONENSE,03,COARI,...,1303,130306,6246.968,-61.644,-3.773,8723.0,10211.0,130300600102,"POLYGON ((-62.992051 -4.523313, -62.989618 -4....",2
30,130063,BERURI,BERURI,1300631,AM,13,N,CENTRO AMAZONENSE,03,COARI,...,1303,130306,17250.370,-61.373,-3.898,7650.0,9050.0,130300600631,"POLYGON ((-62.829671 -4.676299, -62.820848 -4....",3023
32,130083,CAAPIRANGA,CAAPIRANGA,1300839,AM,13,N,CENTRO AMAZONENSE,03,COARI,...,1303,130306,9456.584,-61.209,-3.328,7171.0,6489.0,130300600839,"POLYGON ((-62.582298 -3.106742, -62.57379 -3.1...",2850
33,130008,ANAMÃ,ANAMA,1300086,AM,13,N,CENTRO AMAZONENSE,03,COARI,...,1303,130306,2453.976,-61.404,-3.580,6198.0,6957.0,130300600086,"POLYGON ((-62.08746 -3.278741, -62.077387 -3.2...",400
35,130250,MANACAPURU,MANACAPURU,1302504,AM,13,N,CENTRO AMAZONENSE,03,MANAUS,...,1303,130307,7335.415,-60.621,-3.300,58777.0,65577.0,130300702504,"POLYGON ((-61.483107 -2.787378, -61.479513 -2....",6471
36,130080,BORBA,BORBA,1300805,AM,13,N,SUL AMAZONENSE,04,MADEIRA,...,1304,130413,44258.820,-59.594,-4.388,16502.0,23673.0,130401300805,"POLYGON ((-61.294029 -4.712134, -61.289445 -4....",7441
37,130255,MANAQUIRI,MANAQUIRI,1302553,AM,13,N,CENTRO AMAZONENSE,03,MANAUS,...,1303,130307,3967.589,-60.459,-3.428,10670.0,17311.0,130300702553,"POLYGON ((-61.129962 -3.947296, -61.128518 -3....",9056
38,130185,IRANDUBA,IRANDUBA,1301852,AM,13,N,CENTRO AMAZONENSE,03,MANAUS,...,1303,130307,2203.848,-60.186,-3.285,19855.0,26612.0,130300701852,"POLYGON ((-60.98489 -2.885696, -60.983405 -2.8...",581
39,130353,PRESIDENTE FIGUEIREDO,PRESIDENTE FIGUEIREDO,1303536,AM,13,N,CENTRO AMAZONENSE,03,RIO PRETO DA EVA,...,1303,130308,25420.870,-60.025,-2.034,7783.0,10180.0,130300803536,"POLYGON ((-60.963847 -1.309869, -60.963841 -1....",10758


In [85]:
am.to_file('../results/geo/am-lightning.shp', encoding = 'utf-8')

In [132]:
fig = go.Figure()

labels = list(statistics.keys())
values = list(statistics.values())

for l, v in zip(labels, values):
#     p = 'outside' if v < 12000 else 'inside'
    p = 'outside'
    fig.add_trace(
        go.Bar(x=[v], y=[l], text='%s (%s)'%(l, v), 
#                name=l,
               marker=dict(color='#23A9FC', line=dict(color='#000000', width=.1)),
               textposition=p, orientation='h')
    )
    
fig.update_yaxes(showticklabels=False)
fig.update_layout(
    template='plotly_white', 
    showlegend=False,
    margin={'t': 0, 'b': 0, 'l': 0, 'r': 0},
    
)

offline.plot(fig, 'lightnings-cities.png', 
    image_width=1100,
    image='svg')
offline.iplot(fig)